In [1]:
%load_ext autoreload

%autoreload 2

from enum import auto, StrEnum

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import streamlit as st



In [2]:
from datetime import datetime
from enum import auto, StrEnum
import re

from requests_cache import CachedSession

from urllib.parse import quote
from urllib.request import urlopen

import pandas as pd
import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = (10, 10)


class Preset(StrEnum):
    duel = auto()
    team = auto()
    ffa = auto()
    all = auto()


# Cache the data
def get_data(url: str):
    # print(f"Getting data from {url}")
    session = CachedSession("bar_cache", backend="sqlite")
    data = session.get(url).json()
    return data


def get_fresh_data(url: str):
    # print(f"Getting data from {url}")
    session = CachedSession("short_cache", backend="sqlite", expire_after=60)
    data = session.get(url).json()
    return data


# Get user name from user id
def get_user_name(user_id: int):
    name_list = get_data("https://api.bar-rts.com/cached-users")
    user_name = ""
    for user in name_list:
        if user["id"] == user_id:
            user_name = user["username"]
    return user_name


def get_user_id(user_name: str):
    name_list = get_data("https://api.bar-rts.com/cached-users")
    user_id = ""
    for user in name_list:
        if user["username"] == user_name:
            user_id = user["id"]
    return user_id


def process_match_data(match_details_df):
    if match_details_df.empty:
        return match_details_df
    # Get the winning team and count the number of wins
    match = {}
    matches = []

    for _, game in match_details_df.iterrows():
        # print(game)
        for team in game["AllyTeams"]:
            for player in team["Players"]:
                match = {
                    **match,
                    **{
                        "id": team["id"],
                        "userId": player["userId"],
                        "teamId": player["teamId"],
                        "allyTeamId": player["allyTeamId"],
                        "name": player["name"],
                        "faction": player["faction"],
                        "rank": player["rank"],
                        "skillUncertainty": player["skillUncertainty"],
                        "skill": float(re.sub("[^0123456789\.]", "", player["skill"])),
                        "startPos": player["startPos"],
                        "winningTeam": team["winningTeam"],
                        "Map.fileName": game["Map.fileName"],
                        # + "_"
                        # + str(len(team["Players"]))
                        # + "v"
                        # + str(len(team["Players"])),
                        "Map.scriptName": game["Map.scriptName"],
                        "durationMs": game["durationMs"],
                        "startTime": game["startTime"],
                    },
                }
                matches.append(match)

    matches_df = pd.json_normalize(matches)
    matches_df["startTime"] = pd.to_datetime(matches_df["startTime"])
    return matches_df


# Get match details for each match
def get_match_details(id: str):
    match_details = get_data(f"https://api.bar-rts.com/replays/{id}")

    match_details_df = pd.json_normalize(match_details)
    match_details_df["startTime"] = pd.to_datetime(match_details_df["startTime"])
    return match_details_df


# Get the players replays metadata
def get_match_data(
    matches_bar,
    user: str,
    preset: Preset = Preset.team,
    season0: bool = False,
):
    # Depending on the preset, the API returns different data.json
    # The preset uses the following format: &preset=duel%2Cffa%2Cteam
    # duel%2Cffa%2Cteam is the same as duel,ffa,team

    if preset == Preset.all:
        preset = "&preset=duel&preset=ffa&preset=team"
    else:
        preset = f"&preset={preset.name}"

    # If season0 is true, then only get games after June 1st, 2023 %2C
    date_range = ""
    if season0:
        date_range = f"&date=2023-06-01&date={datetime.today().strftime('%Y-%m-%d')}"

    uri = f"https://api.bar-rts.com/replays?page=1&limit=9999{preset}{date_range}&hasBots=false&endedNormally=true&players="

    data = get_data(f"{uri}{quote(user)}")

    # Test data has attribute data
    # if not hasattr(data, "data"):
    #     return pd.DataFrame()

    # Get the winning team and count the number of wins
    matches = []
    percent_complete = 0
    number_of_matches = len(data["data"])
    for index, game in enumerate(data["data"]):
        percent_complete += 1
        matches_bar.progress(
            percent_complete / number_of_matches,
            text=f"Getting API for game {index+1} out of {number_of_matches} games. ({game['id']})",
        )
        if game["Map"]["fileName"] is not None:
            matches.append(get_match_details(game["id"]))

    if len(matches) == 0:
        return pd.DataFrame()

    matches_df = pd.concat(matches, axis=0)
    return matches_df


# Get the win rate for each map
def get_win_rate(
    df,
    user: str,
    min_games: int = 5,
):
    if df.empty:
        return df

    user_id = get_user_id(user)
    if user_id == "":
        print(f"{user} does not exist")
        return pd.DataFrame()

    win_rate_df = (
        df.query(f"userId == {user_id}")
        .groupby(["Map.fileName"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
        .query(f"count >= {str(min_games)}")
        .sort_values([("mean"), ("count")], ascending=True)
    )
    return win_rate_df


def get_fractions_win_rate(df, user: str):
    return (
        df.query(f"userId == {get_user_id(user)}")
        .groupby(["faction"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
    )


def get_best_teammates(df, user: str, min_games: int = 5):
    team_winrate = df.groupby(  # .query(f"userId == {get_user_id('furyhawk')}")
        ["allyTeamId", "userId"]
    ).agg({"winningTeam": ["mean", "count"]})["winningTeam"]

    # From team_winrate, get the allyTeamId of the user
    team_ally_id = team_winrate.query(f"userId == {get_user_id(user)}").index.unique(
        level="allyTeamId"
    )
    teams_df = df[df["allyTeamId"].isin(team_ally_id)]
    best_teammates_df = (
        teams_df.groupby(["name", "userId"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
        .query(f"count >= {str(min_games)} & userId != {get_user_id(user)}")
        .sort_values([("mean"), ("count")], ascending=False)
    )
    return best_teammates_df


def get_battle_list():
    battles_json = get_fresh_data("https://api.bar-rts.com/battles")
    return pd.json_normalize(battles_json)


def get_battle_details(battles_df):
    battle_list = []

    best_battle = battles_df.head(1)
    for players in best_battle["players"]:
        for player in players:
            if "teamId" in player:  # skill userId
                battle_list.append(
                    {
                        "teamId": player["teamId"],
                        "username": player["username"],
                        "userId": player["userId"],
                        "skill": float(re.sub("[^0123456789\.]", "", player["skill"])),
                        "gameStatus": player["gameStatus"],
                        "map": best_battle["mapFileName"].values[0],
                        "title": best_battle["title"].values[0],
                    }
                )

    return pd.DataFrame(battle_list).sort_values(by="teamId")


def get_map_win_rate(win_rate_df, map_name: str):
    win_rate_df = win_rate_df.reset_index()
    return win_rate_df.loc[win_rate_df["Map.fileName"] == map_name]


In [3]:

plt.rcParams["figure.figsize"] = (10, 10)


class Preset(StrEnum):
    """Preset for the API call"""

    duel = auto()
    team = auto()
    ffa = auto()
    all = auto()


# Set the x axis minor locator to 5 and major locator to 10
# Set the y axis to the map name
def plot_win_rate(
    win_rate_df,
    user: str,
    preset=Preset.team,
):
    # Get overall win rate
    overall_win_rate = win_rate_df["mean"].mean()
    # Get total number of games
    total_games = win_rate_df["count"].sum()

    fig, (ax1, ax2) = plt.subplots(1, 2, sharey="all", figsize=(12, 6))

    bars = ax1.barh(
        y=win_rate_df.index,
        width=win_rate_df["mean"],
        alpha=0.75,
    )
    ax1.bar_label(bars, fmt="{:.0%}", label_type="center")
    ax1.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    ax1.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax1.grid(which="minor", axis="x", linestyle="--")
    ax1.grid(which="major", axis="x", linestyle="-")
    ax1.set_xlabel("Winning %")
    ax1.set_ylabel("Map")
    ax1.set_xlim(0, 1)
    ax1.set_title(f"[{preset.name} games]{user} Winning {overall_win_rate:.0%} by Map")

    ax1.set_axisbelow(True)

    bars = ax2.barh(
        y=win_rate_df.index,
        width=win_rate_df["count"],
        alpha=0.75,
    )

    ax2.bar_label(bars, label_type="center")
    ax2.xaxis.set_minor_locator(ticker.MultipleLocator(5))
    ax2.xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax2.grid(which="minor", axis="x", linestyle="--")
    ax2.grid(which="major", axis="x", linestyle="-")
    ax2.set_xlabel("Games")
    ax2.set_title(f"{user} {total_games} games by Map")

    plt.subplots_adjust(wspace=0, hspace=0)

    return fig

In [4]:
preset = Preset.team
season0: bool = True
user = "furyhawk"

In [5]:
progress_text = "Operation in progress. Please wait."
matches_bar = st.progress(0, text=progress_text)

2023-07-27 10:01:16.856 
  command:

    streamlit run /home/furyhawk/.conda/envs/web/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [6]:
df = process_match_data(get_match_data(matches_bar, user, preset, season0))

In [7]:
df

,id,userId,teamId,allyTeamId,name,faction,rank,skillUncertainty,skill,winningTeam,Map.fileName,Map.scriptName,durationMs,startTime,startPos.x,startPos.y,startPos.z
0,772278,91339,5,772278,suumpmolk,Cortex,3,3,27.20,False,eight_horses_1.2,Eight Horses 1.2,1386533,2023-07-26 15:53:02+00:00,11683.545898,115.925232,6056.775879
1,772278,49050,4,772278,Firefly123,Cortex,5,3,32.80,False,eight_horses_1.2,Eight Horses 1.2,1386533,2023-07-26 15:53:02+00:00,11466.384766,101.799927,7131.794922
2,772278,80622,2,772278,[LOB]Choice,Armada,6,1,39.24,False,eight_horses_1.2,Eight Horses 1.2,1386533,2023-07-26 15:53:02+00:00,11327.764648,104.072754,9841.516602
3,772278,31861,3,772278,RebelNode,Armada,5,3,36.03,False,eight_horses_1.2,Eight Horses 1.2,1386533,2023-07-26 15:53:02+00:00,11531.600586,158.958008,243.316467
4,772278,118006,6,772278,Thembee,Armada,3,3,27.20,False,eight_horses_1.2,Eight Horses 1.2,1386533,2023-07-26 15:53:02+00:00,11901.693359,127.445312,5211.819336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804,616367,96033,12,616367,lolfindkeinnamen,Cortex,2,3,16.17,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,4535.524902,59.813599,7434.944824
1805,616367,83880,13,616367,Ambassador,Armada,3,3,14.79,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,3944.423584,62.832031,9278.013672
1806,616367,88793,10,616367,ThePasst,Armada,3,3,18.00,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,972.909546,265.818604,10482.109375
1807,616367,19953,9,616367,PaniniPie,Cortex,5,3,20.45,True,supreme_strait_v1.4,Supreme Strait v1.4,2066500,2023-06-01 14:28:39+00:00,4863.265137,55.085083,8604.011719


In [8]:
# Get the players replays metadata
def get_quick_match_data(
    user: str,
    preset: Preset = Preset.team,
    season0: bool = False,
):
    # Depending on the preset, the API returns different data.json
    # The preset uses the following format: &preset=duel%2Cffa%2Cteam
    # duel%2Cffa%2Cteam is the same as duel,ffa,team

    if preset == Preset.all:
        preset = "&preset=duel&preset=ffa&preset=team"
    else:
        preset = f"&preset={preset.name}"

    # If season0 is true, then only get games after June 1st, 2023 %2C
    date_range = ""
    if season0:
        date_range = f"&date=2023-06-01&date={datetime.today().strftime('%Y-%m-%d')}"

    uri = f"https://api.bar-rts.com/replays?page=1&limit=9999{preset}{date_range}&hasBots=false&endedNormally=true&players="

    data = get_fresh_data(f"{uri}{quote(user)}")

    # Get the winning team and count the number of wins
    match = {}
    matches = []

    for game in data["data"]:
        if game["Map"]["fileName"] is not None:
            # print(game["id"])
            for team in game["AllyTeams"]:
                for player in team["Players"]:
                    if player["name"] == user:
                        match = {
                            **match,
                            **{
                                "id": game["id"],
                                "name": player["name"],
                                "winningTeam": team["winningTeam"],
                                "Map.fileName": game["Map"]["fileName"],
                                "Map.scriptName": game["Map"]["scriptName"],
                                "durationMs": game["durationMs"],
                                "startTime": game["startTime"],
                            },
                        }
                        matches.append(match)

    matches_df = pd.json_normalize(matches)
    matches_df["startTime"] = pd.to_datetime(matches_df["startTime"])
    return matches_df


In [9]:
win_rate_df = get_quick_match_data(user, preset)

In [10]:
# Get the win rate for each map
def get_quick_win_rate(
    df,
    user: str,
    min_games: int = 5,
):
    if df.empty:
        return df

    win_rate_df = (
        df.groupby(["Map.fileName"])
        .agg({"winningTeam": ["mean", "count"]})["winningTeam"]
        .query(f"count >= {str(min_games)}")
        .sort_values([("mean"), ("count")], ascending=True)
    )
    return win_rate_df


In [11]:
win_rate_df

,id,name,winningTeam,Map.fileName,Map.scriptName,durationMs,startTime
0,5e41c16415691f94b3fb1382e578e8b9,furyhawk,True,eight_horses_1.2,Eight Horses 1.2,1386533,2023-07-26 15:53:02+00:00
1,bac9c064eae1cf099ac0f66a4c6ebe53,furyhawk,False,dsdr_4.1,DSDR 4.1,1806033,2023-07-26 07:22:34+00:00
2,19b0c0647730278d1b9e5861ac6d812c,furyhawk,False,dsdr_4.1,DSDR 4.1,1758900,2023-07-26 05:33:13+00:00
3,ddabc064787476584ec2b6caedbb170d,furyhawk,False,ascendancy_v2.2,Ascendancy v2.2,659833,2023-07-26 05:15:09+00:00
4,7f8cc064a5dddb6751b706808c801740,furyhawk,False,starwatcher_1.0,Starwatcher 1.0,1509133,2023-07-26 03:01:19+00:00
...,...,...,...,...,...,...,...
174,155a5064a582dc362ae4ac708e074e2a,furyhawk,True,ice_scream_v2.5,Ice Scream v2.5,2213167,2023-05-02 00:32:21+00:00
175,b7d24f642113204e8d20d0ab54ad5044,furyhawk,False,supreme_strait_v1.4,Supreme Strait v1.4,1197000,2023-05-01 14:54:47+00:00
176,67c54f643202a0027285f45ae3cf6736,furyhawk,False,thermal_shock_v1.1,Thermal Shock v1.1,1409767,2023-05-01 13:57:59+00:00
177,77b84f649b657130c907ce491b4fbef0,furyhawk,True,dsdr_4.1,DSDR 4.1,2292467,2023-05-01 13:02:47+00:00


In [12]:
get_quick_win_rate(win_rate_df, user, 5).to_dict("records")

[{'mean': 0.25, 'count': 12},
 {'mean': 0.45454545454545453, 'count': 11},
 {'mean': 0.45454545454545453, 'count': 11},
 {'mean': 0.5, 'count': 6},
 {'mean': 0.5, 'count': 8},
 {'mean': 0.5, 'count': 10},
 {'mean': 0.5185185185185185, 'count': 27},
 {'mean': 0.5652173913043478, 'count': 46},
 {'mean': 0.625, 'count': 8},
 {'mean': 0.7142857142857143, 'count': 7}]

In [35]:
user

'furyhawk'

In [34]:
get_quick_win_rate(win_rate_df, user, 1)

,mean,count
Map.fileName,,
Crescent_Bay_V2,0.000000,1
Nine_Metal_Isles_V1,0.000000,1
Rustcrown Canyon 1.1,0.000000,1
Tempest_V3,0.000000,1
boulder_beach_v1,0.000000,1
comet_catcher_remake_1.8,0.000000,1
eye_of_horus_1.7,0.000000,1
ghenna_rising_4.0,0.000000,1
hotlips_remake_v3.1,0.000000,1


In [31]:
win_rate_user_df = get_quick_win_rate(win_rate_df, user, 5)
df = win_rate_user_df.reset_index()
map_df = df.loc[df["Map.fileName"] == "supreme_strait_v1.4"]
records = map_df.to_dict("records")[-1]

In [32]:
df

,Map.fileName,mean,count
0,ice_scream_v2.5,0.285714,7
1,all_that_glitters_v2.0,0.400000,5
2,forge_v1.4,0.400000,10
3,koom_valley_3_3.1,0.416667,12
4,supreme_strait_v1.4,0.488189,127
5,all_that_glitters_1.2,0.500000,8
6,flats_and_forests_v2.1,0.500000,20
7,ascendancy_v2.0,0.538462,13
8,all_that_glitters_v2.1,0.539326,89
9,darkside_v3.0,0.555556,9


In [14]:
records

{'Map.fileName': 'supreme_strait_v1.4',
 'mean': 0.5652173913043478,
 'count': 46}

In [15]:
battles_df = get_battle_list()
battle_detail_df = get_battle_details(battles_df)
team_size = len(battle_detail_df.index)
battle_detail_df.head(team_size // 2)
battle_detail_df.tail(team_size // 2)

,teamId,username,userId,skill,gameStatus,map,title
13,9,SteakTartar,1813,38.94,Playing,swirly_rock_v1.1,EU - 2 01
12,10,Steel_Blue,16298,38.19,Playing,swirly_rock_v1.1,EU - 2 01
15,11,[TOXIC]Fruit,63932,37.47,Playing,swirly_rock_v1.1,EU - 2 01
11,12,Dylon,67133,33.85,Playing,swirly_rock_v1.1,EU - 2 01
14,13,bingbingbang,180,29.40,Playing,swirly_rock_v1.1,EU - 2 01
0,14,SuperLux,51141,27.71,Playing,swirly_rock_v1.1,EU - 2 01
8,15,Torkle,108936,25.46,Playing,swirly_rock_v1.1,EU - 2 01
2,16,Fatpanda,60716,23.83,Playing,swirly_rock_v1.1,EU - 2 01


In [16]:
battle_detail_df

,teamId,username,userId,skill,gameStatus,map,title
7,1,kyouma001,98291,43.22,Playing,swirly_rock_v1.1,EU - 2 01
5,2,avocado,101921,42.68,Playing,swirly_rock_v1.1,EU - 2 01
3,3,APolygon,26299,32.99,Playing,swirly_rock_v1.1,EU - 2 01
1,4,garbager,26301,31.22,Playing,swirly_rock_v1.1,EU - 2 01
9,5,Klu,37619,30.00,Playing,swirly_rock_v1.1,EU - 2 01
10,6,PrimalRIP,37618,29.12,Playing,swirly_rock_v1.1,EU - 2 01
4,7,Jarion,58259,24.19,Playing,swirly_rock_v1.1,EU - 2 01
6,8,Chaz706,35123,23.35,Playing,swirly_rock_v1.1,EU - 2 01
13,9,SteakTartar,1813,38.94,Playing,swirly_rock_v1.1,EU - 2 01
12,10,Steel_Blue,16298,38.19,Playing,swirly_rock_v1.1,EU - 2 01


In [17]:
battle_list = []

for _, player in battle_detail_df.iterrows():
    battle = {}
    win_rate_df = get_quick_match_data(player["username"], preset)
    win_rate_user_df = get_quick_win_rate(win_rate_df, 1)
    df = win_rate_user_df.reset_index()
    map_df = df.loc[df["Map.fileName"] == player["map"]]
    if not map_df.empty:
        record = map_df.to_dict("records")[-1]
        battle = {
            "teamId": player["teamId"],
            "userId": player["userId"],
            "username": player["username"],
            "skill": player["skill"],
            "gameStatus": player["gameStatus"],
            **record,
        }
    else:
        battle = {
            "username": player["username"],
            "userId": player["userId"],
            "username": player["username"],
            "skill": player["skill"],
            "gameStatus": player["gameStatus"],
            "Map.fileName": player["map"],
            "mean": 0.5,
            "count": 0,
        }
    battle_list.append(battle)
pd.DataFrame(battle_list)

,username,userId,skill,gameStatus,Map.fileName,mean,count
0,kyouma001,98291,43.22,Playing,swirly_rock_v1.1,0.5,0
1,avocado,101921,42.68,Playing,swirly_rock_v1.1,0.5,0
2,APolygon,26299,32.99,Playing,swirly_rock_v1.1,0.5,0
3,garbager,26301,31.22,Playing,swirly_rock_v1.1,0.5,0
4,Klu,37619,30.00,Playing,swirly_rock_v1.1,0.5,0
5,PrimalRIP,37618,29.12,Playing,swirly_rock_v1.1,0.5,0
6,Jarion,58259,24.19,Playing,swirly_rock_v1.1,0.5,0
7,Chaz706,35123,23.35,Playing,swirly_rock_v1.1,0.5,0
8,SteakTartar,1813,38.94,Playing,swirly_rock_v1.1,0.5,0
9,Steel_Blue,16298,38.19,Playing,swirly_rock_v1.1,0.5,0


In [18]:
pd.DataFrame(battle_list)

,username,userId,skill,gameStatus,Map.fileName,mean,count
0,kyouma001,98291,43.22,Playing,swirly_rock_v1.1,0.5,0
1,avocado,101921,42.68,Playing,swirly_rock_v1.1,0.5,0
2,APolygon,26299,32.99,Playing,swirly_rock_v1.1,0.5,0
3,garbager,26301,31.22,Playing,swirly_rock_v1.1,0.5,0
4,Klu,37619,30.00,Playing,swirly_rock_v1.1,0.5,0
5,PrimalRIP,37618,29.12,Playing,swirly_rock_v1.1,0.5,0
6,Jarion,58259,24.19,Playing,swirly_rock_v1.1,0.5,0
7,Chaz706,35123,23.35,Playing,swirly_rock_v1.1,0.5,0
8,SteakTartar,1813,38.94,Playing,swirly_rock_v1.1,0.5,0
9,Steel_Blue,16298,38.19,Playing,swirly_rock_v1.1,0.5,0


In [19]:
battle_win_rate_df = pd.DataFrame(battle_list)

In [20]:
team_size = len(battle_win_rate_df.index)
team1_df = battle_win_rate_df.head(team_size // 2)


In [21]:
team1_df

,username,userId,skill,gameStatus,Map.fileName,mean,count
0,kyouma001,98291,43.22,Playing,swirly_rock_v1.1,0.5,0
1,avocado,101921,42.68,Playing,swirly_rock_v1.1,0.5,0
2,APolygon,26299,32.99,Playing,swirly_rock_v1.1,0.5,0
3,garbager,26301,31.22,Playing,swirly_rock_v1.1,0.5,0
4,Klu,37619,30.00,Playing,swirly_rock_v1.1,0.5,0
5,PrimalRIP,37618,29.12,Playing,swirly_rock_v1.1,0.5,0
6,Jarion,58259,24.19,Playing,swirly_rock_v1.1,0.5,0
7,Chaz706,35123,23.35,Playing,swirly_rock_v1.1,0.5,0


In [22]:
team1_df.agg({"mean": ["mean", "count"]})["mean"]

mean     0.5
count    8.0
Name: mean, dtype: float64

In [23]:
team1_df["skill"].sum()

256.77000000000004

In [24]:
team2_df = battle_win_rate_df.tail(team_size // 2)

In [25]:
team2_df

,username,userId,skill,gameStatus,Map.fileName,mean,count
8,SteakTartar,1813,38.94,Playing,swirly_rock_v1.1,0.5,0
9,Steel_Blue,16298,38.19,Playing,swirly_rock_v1.1,0.5,0
10,[TOXIC]Fruit,63932,37.47,Playing,swirly_rock_v1.1,0.5,0
11,Dylon,67133,33.85,Playing,swirly_rock_v1.1,0.5,0
12,bingbingbang,180,29.40,Playing,swirly_rock_v1.1,0.5,0
13,SuperLux,51141,27.71,Playing,swirly_rock_v1.1,0.5,0
14,Torkle,108936,25.46,Playing,swirly_rock_v1.1,0.5,0
15,Fatpanda,60716,23.83,Playing,swirly_rock_v1.1,0.5,0


In [36]:
import numpy as np

np.average(team2_df["mean"], weights=team2_df["skill"])

0.5

In [26]:
team2_df.agg({"mean": ["mean", "count"]})["mean"][0]

0.5

In [27]:
team2_df["skill"].sum()

254.85

In [28]:
win_rate_user_df

,mean,count
Map.fileName,,
ice_scream_v2.5,0.285714,7
all_that_glitters_v2.0,0.400000,5
forge_v1.4,0.400000,10
koom_valley_3_3.1,0.416667,12
supreme_strait_v1.4,0.488189,127
all_that_glitters_1.2,0.500000,8
flats_and_forests_v2.1,0.500000,20
ascendancy_v2.0,0.538462,13
all_that_glitters_v2.1,0.539326,89


In [29]:
battle_list

[{'username': 'kyouma001',
  'userId': 98291,
  'skill': 43.22,
  'gameStatus': 'Playing',
  'Map.fileName': 'swirly_rock_v1.1',
  'mean': 0.5,
  'count': 0},
 {'username': 'avocado',
  'userId': 101921,
  'skill': 42.68,
  'gameStatus': 'Playing',
  'Map.fileName': 'swirly_rock_v1.1',
  'mean': 0.5,
  'count': 0},
 {'username': 'APolygon',
  'userId': 26299,
  'skill': 32.99,
  'gameStatus': 'Playing',
  'Map.fileName': 'swirly_rock_v1.1',
  'mean': 0.5,
  'count': 0},
 {'username': 'garbager',
  'userId': 26301,
  'skill': 31.22,
  'gameStatus': 'Playing',
  'Map.fileName': 'swirly_rock_v1.1',
  'mean': 0.5,
  'count': 0},
 {'username': 'Klu',
  'userId': 37619,
  'skill': 30.0,
  'gameStatus': 'Playing',
  'Map.fileName': 'swirly_rock_v1.1',
  'mean': 0.5,
  'count': 0},
 {'username': 'PrimalRIP',
  'userId': 37618,
  'skill': 29.12,
  'gameStatus': 'Playing',
  'Map.fileName': 'swirly_rock_v1.1',
  'mean': 0.5,
  'count': 0},
 {'username': 'Jarion',
  'userId': 58259,
  'skill': 24

In [30]:
win_rate_user_df

,mean,count
Map.fileName,,
ice_scream_v2.5,0.285714,7
all_that_glitters_v2.0,0.400000,5
forge_v1.4,0.400000,10
koom_valley_3_3.1,0.416667,12
supreme_strait_v1.4,0.488189,127
all_that_glitters_1.2,0.500000,8
flats_and_forests_v2.1,0.500000,20
ascendancy_v2.0,0.538462,13
all_that_glitters_v2.1,0.539326,89
